In [1]:
# HIDE CODE


import os
import sys
import nibabel as nib
import networkx as nx
from time import time
from pprint import pprint
from copy import deepcopy as dc
from os.path import join as pjoin
from myterial import orange, blue_grey
from IPython.display import display, IFrame, HTML
from matplotlib.colors import rgb2hex, to_rgb
import matplotlib.pyplot as plt
import seaborn as sns

# set style & no interpolalation
import matplotlib
matplotlib.rcParams['image.interpolation'] = 'none'
sns.set_style('whitegrid')

# tmp & extras dir
git_dir = pjoin(os.environ['HOME'], 'Dropbox/git')
tmp_dir = pjoin(git_dir, 'jb-Ca-fMRI/tmp')
extras_dir = pjoin(git_dir, 'jb-Ca-fMRI/_extras')

# GitHub
sys.path.insert(0, pjoin(git_dir, '_Ca-fMRI'))
from register.atlas import load_allen, make_tree_graph
from register.parcellation import Parcellation
from analysis.hierarchical import Hierarchical
from analysis.svinet import *
from utils.plotting import *
from utils.render import *
from model.mouse import Mice
from model.configuration import Config

# warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
naming_key = {
    'BIS_output_SLC01': 'animal01',
    'BIS_output_SLC06': 'animal02',
    'BIS_output_SLC04': 'animal03',
    'BIS_output_SLC03': 'animal04',
    'BIS_output_SLC05': 'animal05',
    'BIS_output_SLC02': 'animal06',
    'BIS_output_SLC07': 'animal55',
    'BIS_output_SLC09': 'animal56',
    'BIS_output_SLC08': 'animal57',
    'BIS_output_SLC10': 'animal73',
}
naming_key = {y: x for x, y in naming_key.items()}
src_dir = '/home/hadi/Documents/hv88/stim_files'
dst_dir = '/mnt/storage/hadi/Ca-fMRI/stim'
os.makedirs(dst_dir, exist_ok=True)

In [3]:
src2dst = {}
for src_file in os.listdir(src_dir):
    _, sub, ses, _, run, task = src_file.split('_')
    tgt_file = [
        f"sub-{naming_key[sub].split('_')[-1]}",
        ses,
        f"run-{int(run[3])}",
        f"task-{task.split('.')[0].lower()}",
        'desc-stim',
    ]
    dst_file = '_'.join(tgt_file)
    src2dst[src_file] = f"{dst_file}.csv"
src2dst = dict(sorted(src2dst.items(), key=lambda x: x[1]))

In [4]:
src2dst

{
    'SLC_animal01_ses-1_2019-01-09_EPI2_LED.csv': 'sub-SLC01_ses-1_run-2_task-led_desc-stim.csv',
    'SLC_animal01_ses-1_2019-01-09_EPI4_LED.csv': 'sub-SLC01_ses-1_run-4_task-led_desc-stim.csv',
    'SLC_animal01_ses-1_2019-01-09_EPI6_LED.csv': 'sub-SLC01_ses-1_run-6_task-led_desc-stim.csv',
    'SLC_animal01_ses-2_2019-01-18_EPI2_LED.csv': 'sub-SLC01_ses-2_run-2_task-led_desc-stim.csv',
    'SLC_animal01_ses-2_2019-01-18_EPI3_REST.csv': 'sub-SLC01_ses-2_run-3_task-rest_desc-stim.csv',
    'SLC_animal01_ses-2_2019-01-18_EPI4_LED.csv': 'sub-SLC01_ses-2_run-4_task-led_desc-stim.csv',
    'SLC_animal01_ses-2_2019-01-18_EPI5_REST.csv': 'sub-SLC01_ses-2_run-5_task-rest_desc-stim.csv',
    'SLC_animal01_ses-2_2019-01-18_EPI6_LED.csv': 'sub-SLC01_ses-2_run-6_task-led_desc-stim.csv',
    'SLC_animal01_ses-2_2019-01-18_EPI7_REST.csv': 'sub-SLC01_ses-2_run-7_task-rest_desc-stim.csv',
    'SLC_animal01_ses-3_2019-01-23_EPI2_LED.csv': 'sub-SLC01_ses-3_run-2_task-led_desc-stim.csv',
    'SLC_animal01_ses-3_2019-01-23_EPI4_LED.csv': 'sub-SLC01_ses-3_run-4_task-led_desc-stim.csv',
    'SLC_animal01_ses-3_2019-01-23_EPI6_LED.csv': 'sub-SLC01_ses-3_run-6_task-led_desc-stim.csv',
    'SLC_animal01_ses-3_2019-01-23_EPI7_REST.csv': 'sub-SLC01_ses-3_run-7_task-rest_desc-stim.csv',
    'SLC_animal06_ses-1_2019-01-10_EPI2_LED.csv': 'sub-SLC02_ses-1_run-2_task-led_desc-stim.csv',
    'SLC_animal06_ses-1_2019-01-10_EPI3_REST.csv': 'sub-SLC02_ses-1_run-3_task-rest_desc-stim.csv',
    'SLC_animal06_ses-1_2019-01-10_EPI4_LED.csv': 'sub-SLC02_ses-1_run-4_task-led_desc-stim.csv',
    'SLC_animal06_ses-1_2019-01-10_EPI5_REST.csv': 'sub-SLC02_ses-1_run-5_task-rest_desc-stim.csv',
    'SLC_animal06_ses-1_2019-01-10_EPI6_LED.csv': 'sub-SLC02_ses-1_run-6_task-led_desc-stim.csv',
    'SLC_animal06_ses-2_2019-01-17_EPI2_LED.csv': 'sub-SLC02_ses-2_run-2_task-led_desc-stim.csv',
    'SLC_animal06_ses-2_2019-01-17_EPI4_LED.csv': 'sub-SLC02_ses-2_run-4_task-led_desc-stim.csv',
    'SLC_animal06_ses-2_2019-01-17_EPI6_LED.csv': 'sub-SLC02_ses-2_run-6_task-led_desc-stim.csv',
    'SLC_animal06_ses-2_2019-01-17_EPI7_LED.csv': 'sub-SLC02_ses-2_run-7_task-led_desc-stim.csv',
    'SLC_animal06_ses-3_2019-01-22_EPI2_LED.csv': 'sub-SLC02_ses-3_run-2_task-led_desc-stim.csv',
    'SLC_animal06_ses-3_2019-01-22_EPI3_REST.csv': 'sub-SLC02_ses-3_run-3_task-rest_desc-stim.csv',
    'SLC_animal06_ses-3_2019-01-22_EPI4_LED.csv': 'sub-SLC02_ses-3_run-4_task-led_desc-stim.csv',
    'SLC_animal06_ses-3_2019-01-22_EPI6_LED.csv': 'sub-SLC02_ses-3_run-6_task-led_desc-stim.csv',
    'SLC_animal04_ses-1_2019-01-10_EPI2_REST.csv': 'sub-SLC03_ses-1_run-2_task-rest_desc-stim.csv',
    'SLC_animal04_ses-1_2019-01-10_EPI4_REST.csv': 'sub-SLC03_ses-1_run-4_task-rest_desc-stim.csv',
    'SLC_animal04_ses-1_2019-01-10_EPI6_REST.csv': 'sub-SLC03_ses-1_run-6_task-rest_desc-stim.csv',
    'SLC_animal04_ses-1_2019-01-10_EPI7_LED.csv': 'sub-SLC03_ses-1_run-7_task-led_desc-stim.csv',
    'SLC_animal04_ses-2_2019-01-17_EPI2_LED.csv': 'sub-SLC03_ses-2_run-2_task-led_desc-stim.csv',
    'SLC_animal04_ses-2_2019-01-17_EPI4_LED.csv': 'sub-SLC03_ses-2_run-4_task-led_desc-stim.csv',
    'SLC_animal04_ses-2_2019-01-17_EPI6_LED.csv': 'sub-SLC03_ses-2_run-6_task-led_desc-stim.csv',
    'SLC_animal04_ses-2_2019-01-17_EPI7_REST.csv': 'sub-SLC03_ses-2_run-7_task-rest_desc-stim.csv',
    'SLC_animal04_ses-3_2019-01-23_EPI2_LED.csv': 'sub-SLC03_ses-3_run-2_task-led_desc-stim.csv',
    'SLC_animal04_ses-3_2019-01-23_EPI4_LED.csv': 'sub-SLC03_ses-3_run-4_task-led_desc-stim.csv',
    'SLC_animal04_ses-3_2019-01-23_EPI6_LED.csv': 'sub-SLC03_ses-3_run-6_task-led_desc-stim.csv',
    'SLC_animal03_ses-1_2019-01-09_EPI2_LED.csv': 'sub-SLC04_ses-1_run-2_task-led_desc-stim.csv',
    'SLC_animal03_ses-1_2019-01-09_EPI4_LED.csv': 'sub-SLC04_ses-1_run-4_task-led_desc-stim.csv',
    'SLC_animal03_ses-1_2019-01-09_EPI6_LED.csv': 'sub-SLC04_ses-1_run-6_task-led_desc-stim.csv',
    'SLC_animal03_ses-1_2019-01-09_EPI7_REST.csv': 'sub-

In [5]:
for s, d in src2dst.items():
    shutil.copyfile(
        src=pjoin(src_dir, s),
        dst=pjoin(dst_dir, d),
    )

In [6]:
for d in src2dst.values():
    if 'rest' in d:
        f = pjoin(dst_dir, d)
        stim = pd.read_csv(pjoin(dst_dir, d), index_col=0)
        if stim.ledStim10Hz.sum() != 0:
            print(d, stim.ledStim1Hz.sum(), stim.ledStim10Hz.sum())

sub-SLC03_ses-1_run-2_task-rest_desc-stim.csv 50.0 477

sub-SLC03_ses-1_run-4_task-rest_desc-stim.csv 50.0 479

sub-SLC03_ses-1_run-6_task-rest_desc-stim.csv 41.0 475

sub-SLC06_ses-1_run-1_task-rest_desc-stim.csv 0.0 7